In [1]:
def gram_schmidt_columns(X):
    """The gram schmidt transform will provide orthogonal vectors. If the vectors we provide exist in a planar manifold
    the resulting vectors should remain in the manifold. TBC!!!"""
    Q, R = np.linalg.qr(X)
    return Q

def rotate_in_subspace(V,v = None,angleinvert=False):
    #creates a rotation matrix that rotates between two vectors
    #defined as the columns of V,
    #such that the rotation occurs in a plane defined by the two vectors
    #(as two vectors is insufficient to describe an arbitrary rotation in n-dimensions)

    #this used to be a single vector, but to generate a general matrix result:
    if v is None:
        v = np.eye(3) 
    
    #the two vectors now with length 1
    Vnormalised = V*np.repeat(1./np.sqrt(np.sum(V**2,0))[None,:],3,axis=0)

    #this is the same span but with orthonormal vectors (each column is one vector)
    #annoyingly this can swap the direction of a rotation...
    G = gram_schmidt_columns(V)

    #We find the matrix that projects from the space to the hyperplane subspace
    P = np.dot(G,G.T)

    #projected onto the plane
    vprime = np.dot(P,v)

    #the difference between v and vprime
    deltav = vprime-v

    #coord in basis
    coords = np.dot(G.T,vprime)

    #angle of the rotation
    ang = np.arccos(np.dot(Vnormalised[:,0],Vnormalised[:,1]))
    if angleinvert:
        ang = -ang

    #the rotation matrix in the 2d plane
    R = np.array([[np.cos(ang),-np.sin(ang)],[np.sin(ang),np.cos(ang)]])

    #new coordinates on the plane
    newcoords = np.dot(R,coords)

    #reprojected back to the original basis
    vprimerotated =np.dot(G,newcoords)

    #translated back from the plane
    vrotated = vprimerotated-deltav    
    return vrotated

def adjusted_rotate_in_subspace(V,v=None):
    #Temporary fix - there's bound to be better!
    #the problem is when the gram schmidt orthonormal basis is created, the direction of rotation can be swapped.
    #as there's two choices, I simply find out whether we've got the right one...
    
    #we feed in the first vector in...
    new = rotate_in_subspace(V,V[:,0])
    new/=np.sqrt(np.sum(new**2))
    #we expect the result to be the second vector...
    targ=V[:,1]/np.sqrt(np.sum(V[:,1]**2))
    
    #we've normalised them both: They should be equal
    #if they're not...
    if np.sum((new-targ)**2)>0.01: #we need to swap the order of the vectors!
        #print "Flipped angle"
        return rotate_in_subspace(V,v,angleinvert=True)
    else:
        return rotate_in_subspace(V,v)

Rather than all that hassle, why aren't we just using the orthonormal basis provided by the eigenvectors!

In [ ]:
#    while np.any(evals>1.0):
#        indices = np.real(evals)>1.0                
#        e = evects[:,indices][:,0] #we'll just do one...
#        v = evals[indices][0]      
#        Smat = np.eye(len(e))
#        Smat[0]*=v
#        cov = np.dot(evects,np.dot(Smat,np.dot(np.linalg.inv(evects),cov)))
#        #cov += newcov * 0.1        
#        evals,evects = np.linalg.eig(np.dot(np.linalg.inv(cov),newcov))
        